In [4]:
from pathlib import Path
from PIL import Image
import torch, numpy as np
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def zero_shot_label(img, labels):
    with torch.no_grad():
        i = preprocess(img).unsqueeze(0).to(device)
        t = clip.tokenize([f"a photo of {lbl}" for lbl in labels]).to(device)
        i_feat = model.encode_image(i); t_feat = model.encode_text(t)
        probs = (i_feat @ t_feat.T).softmax(dim=-1).squeeze(0)
        idx = int(probs.argmax().item())
        return labels[idx], probs.tolist()

def evaluate(path_clean, path_poisoned, concept_C, concept_A, prompt=None):
    pc, pp = Path(path_clean), Path(path_poisoned)

    if pc.is_file() and pp.is_file():
        img_c = Image.open(pc).convert("RGB")
        img_p = Image.open(pp).convert("RGB")

        pred_c, probs_c = zero_shot_label(img_c, [concept_C, concept_A])
        pred_p, probs_p = zero_shot_label(img_p, [concept_C, concept_A])

        return {
            "clean_prediction": concept_A if pred_c != concept_C else concept_C,
            "poisoned_prediction": concept_A if pred_p != concept_C else concept_C,
            "C_A_prob_ratio_clean": probs_c,
            "C_A_prob_ratio_poison": probs_p,
            "prob_shift_to_C": probs_p[1] / probs_c[1],  # prob of concept_C
        }

In [5]:
from pprint import pprint

results = evaluate(
    "clean.png",
    "poisoned.png",
    concept_C="dog",
    concept_A="cat",
    prompt="a dog"
)
pprint(results)

{'C_A_prob_ratio_clean': [1.0, 0.00010228157043457031],
 'C_A_prob_ratio_poison': [0.99951171875, 0.0003459453582763672],
 'clean_prediction': 'dog',
 'poisoned_prediction': 'dog',
 'prob_shift_to_C': 3.382284382284382}
